### k3d-util.pyを同じディレクトリに格納しておくこと。

In [ ]:
import os, glob, time
import numpy as np
import cv2
from PIL import Image
# Image.MAX_IMAGE_PIXELS = 2560*2160*10    # 画像サイズが大きいときはここの数値を大きくする

In [ ]:
import k3d

In [ ]:
import k3d_util as util

In [ ]:
# データの格納場所
data_path = './tutorial_spheroid.npz'

# 対物レンズ倍率
util.MAGNIFICATION = 40

# 撮影時のビニング
util.BINNING = 1

# Z間隔
util.Z_SLICE_LENGTH = 2.0

# 画像の画素数を減らすための画像縮小率
util.REDUCE_RATIO = 4

# Zスタックの枚数を減らすためのZステップ間隔
util.Z_STEP = 1

In [ ]:
# 3D画像データの読み込み関数の定義
def load_data(file_list):
    file_list.sort()
    img = Image.open(file_list[0])
    buf = np.zeros((len(file_list[::util.Z_STEP]), img.size[1]//util.REDUCE_RATIO, img.size[0]//util.REDUCE_RATIO), dtype=np.float32)
    for z, i in enumerate(file_list[::util.Z_STEP]):
        img = Image.open(i)
        img = np.asarray(img, dtype=np.float32)
        # y軸の正方向が画面奥になるため(通常2D画像はy軸の正方向は下)、::-1で行を反転する
        img = cv2.resize(img[::-1, :], (buf.shape[2], buf.shape[1]), interpolation=cv2.INTER_AREA)
        buf[z, :, :] = img
    return buf

In [ ]:
#チャネルごとに3D画像データを読み込む
#data1 = load_data(glob.glob(os.path.join(data_path, 'W0040*C1.tif')))
#data2 = load_data(glob.glob(os.path.join(data_path, 'W0040*C2.tif')))
#np.savez_compressed(data_path, data1=data1, data2=data2)
saved_data = np.load(data_path)
data_list = [saved_data['data1'], saved_data['data2']]

In [ ]:
# 3D画像データの寸法を計算する
util.init_bounds(data_list[0])

In [ ]:
# volume_listに画像データ(複数チャネル)
obj = k3d.multi_mip(
    volume_list=data_list, 
    color_map_list=util.COLOR_MAP_LIST,
    samples=0, 
    gradient_step=0, 
    alpha_blending=True,
    bounds=(0, util.cx, 0, util.cy, 0, util.cz)
)

In [ ]:
# 3Dビューアの表示エリアを生成
plot = k3d.plot(background_color=0x000000, grid_visible=False, lighting=0, height=540)
plot += obj

In [ ]:
# コントロールの準備
util.init_controls(plot, obj, phi=40, distance=(1000, 1, 3000))
fov = 20

### 次のセルを実行したら、3D表示が完了するまでその次のセルを実行しないこと。

In [ ]:
# 3Dビューア及びコントロールの表示
plot.display()
util.display_controls() 

# コントロールの初期値を変更
util.slider_ch[0].value = [172, 900]
util. slider_ch[1].value = [120, 500]

In [ ]:
# FOVやカメラ位置を3D表示に反映する (いったん3D表示してからでないと働かないことがあるので、ここで実行する)
util.refresh()
plot.camera_fov = fov

### 動画作成はこちら👇
* 動画作成にはscikit-video・frame_writer2.pyが必要。

In [ ]:
util.display_movie_controls()

In [ ]:
util.input_movie_filename.value = 'tutorial_spheroid_30fps.mp4'

### 以下はシーケンスデータのサンプル

In [ ]:
util.state_store=[{'duration': 0,
  'camera_pos': (0,0,0,0,0,0,0,0,0),
  'ox': 0.0,
  'oy': 0.0,
  'oz': 0.0,
  'axis': 'z',
  'ha': 0.0,
  'va': 80.0,
  'd': 200.0,
  'check_ch': [True, True],
  'slider_ch': [(172, 900), (120, 500)],
  'plane_x': (0, 640),
  'plane_y': (0, 540),
  'plane_z': (0, 1),
  'alpha': False},
 {'duration': 6,
  'camera_pos': (0,0,0,0,0,0,0,0,0),
  'ox': 0.0,
  'oy': 0.0,
  'oz': 0.0,
  'axis': 'z',
  'ha': 0.0,
  'va': 1.0,
  'd': 1000.0,
  'check_ch': [True, True],
  'slider_ch': [(172, 900), (120, 500)],
  'plane_x': (0, 640),
  'plane_y': (0, 540),
  'plane_z': (0, 16),
  'alpha': False},
 {'duration': 1,
  'camera_pos': (0,0,0,0,0,0,0,0,0),
  'ox': 0.0,
  'oy': 0.0,
  'oz': 0.0,
  'axis': 'z',
  'ha': 0.0,
  'va': 1.0,
  'd': 1000.0,
  'check_ch': [True, True],
  'slider_ch': [(172, 900), (120, 1481)],
  'plane_x': (0, 640),
  'plane_y': (0, 540),
  'plane_z': (0, 16),
  'alpha': False},
 {'duration': 1,
  'camera_pos': (0,0,0,0,0,0,0,0,0),
  'ox': 0.0,
  'oy': 0.0,
  'oz': 0.0,
  'axis': 'z',
  'ha': 0.0,
  'va': 1.0,
  'd': 1000.0,
  'check_ch': [True, True],
  'slider_ch': [(172, 900), (120, 500)],
  'plane_x': (0, 640),
  'plane_y': (0, 540),
  'plane_z': (0, 16),
  'alpha': False},
 {'duration': 1,
  'camera_pos': (0,0,0,0,0,0,0,0,0),
  'ox': 0.0,
  'oy': 0.0,
  'oz': 0.0,
  'axis': 'z',
  'ha': 0.0,
  'va': 1.0,
  'd': 1000.0,
  'check_ch': [True, True],
  'slider_ch': [(172, 2036), (120, 500)],
  'plane_x': (0, 640),
  'plane_y': (0, 540),
  'plane_z': (0, 16),
  'alpha': False},
 {'duration': 1,
  'camera_pos': (0,0,0,0,0,0,0,0,0),
  'ox': 0.0,
  'oy': 0.0,
  'oz': 0.0,
  'axis': 'z',
  'ha': 0.0,
  'va': 1.0,
  'd': 1000.0,
  'check_ch': [True, True],
  'slider_ch': [(172, 900), (120, 500)],
  'plane_x': (0, 640),
  'plane_y': (0, 540),
  'plane_z': (0, 16),
  'alpha': False},
 {'duration': 6,
  'camera_pos': (0,0,0,0,0,0,0,0,0),
  'ox': 0.0,
  'oy': 0.0,
  'oz': 0.0,
  'axis': 'z',
  'ha': 270.0,
  'va': 70.0,
  'd': 600.0,
  'check_ch': [True, True],
  'slider_ch': [(172, 900), (120, 500)],
  'plane_x': (0, 640),
  'plane_y': (0, 540),
  'plane_z': (0, 16),
  'alpha': False},
 {'duration': 6,
  'camera_pos': (0,0,0,0,0,0,0,0,0),
  'ox': -261.3,
  'oy': 0.0,
  'oz': 0.0,
  'axis': 'z',
  'ha': 270.0,
  'va': 70.0,
  'd': 600.0,
  'check_ch': [True, True],
  'slider_ch': [(172, 900), (120, 500)],
  'plane_x': (0, 1),
  'plane_y': (0, 540),
  'plane_z': (0, 16),
  'alpha': False},
 {'duration': 6,
  'camera_pos': (0,0,0,0,0,0,0,0,0),
  'ox': 0.0,
  'oy': 0.0,
  'oz': 0.0,
  'axis': 'z',
  'ha': 360.0,
  'va': 1.0,
  'd': 1000.0,
  'check_ch': [True, True],
  'slider_ch': [(172, 900), (120, 500)],
  'plane_x': (0, 640),
  'plane_y': (0, 540),
  'plane_z': (0, 16),
  'alpha': False}]

### スクリーンショットはalpha_blending==Trueのときは正常に得られないので注意。

In [ ]:
plot.fetch_screenshot()

In [ ]:
# ブラウザからサーバのPythonへ転送するのに時間がかかるため、0バイトの場合がある。
len(plot.screenshot)

In [ ]:
import base64, io
png_stream = base64.b64decode(plot.screenshot)
img = Image.open(io.BytesIO(png_stream))
img